In [ ]:
import os
import cv2
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from matplotlib.image import imread
from tensorflow.keras.utils import Sequence
from tensorflow.keras import layers, models
from keras import backend as K
from skimage.measure import label, regionprops
from sklearn.model_selection import train_test_split

# Data Load

In [ ]:
train_segmentations = pd.read_csv('/kaggle/input/airbus-ship-detection/train_ship_segmentations_v2.csv')
sample_submission = pd.read_csv('/kaggle/input/airbus-ship-detection/sample_submission_v2.csv')

train_v2_path = '/kaggle/input/airbus-ship-detection/train_v2'

In [ ]:
def rle_encode(img):
    """
    Encode a binary mask represented as a 2D numpy array using Run-Length Encoding (RLE).

    Parameters:
    - img (numpy.ndarray): A 2D binary array representing the mask.

    Returns:
    - str: The RLE-encoded string representing the binary mask.
    """

    pixels = img.T.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)


def rle_decode(mask_rle, shape=(768, 768)):
    """
    Decode a Run-Length Encoded (RLE) binary mask into a 2D numpy array.

    Parameters:
    - mask_rle (str): The RLE-encoded string representing the binary mask.
    - shape (tuple, optional): The shape of the target 2D array. Default is (768, 768).

    Returns:
    - numpy.ndarray: A 2D binary array representing the decoded mask.
    """

    if type(mask_rle) != str:
        return np.zeros(shape)

    s = mask_rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(shape[0] * shape[1], dtype=np.uint8)

    for lo, hi in zip(starts, ends):
        img[lo:hi] = 1

    return img.reshape(shape).T # Needed to align to RLE direction

In [ ]:
def display_masks(folder_path, csv_file, num_images=5):
    # Get a list of all files in the folder
    image_files = [f for f in os.listdir(folder_path)[:num_images]]

    # Calculate the number of rows needed
    num_rows = 2

    # Display the images in a grid
    fig, axes = plt.subplots(num_rows, num_images, figsize=(15, 6))

    for i in range(num_images):
        img_path = os.path.join(folder_path, image_files[i])
        img = imread(img_path)

        all_masks = csv_file[csv_file['ImageId'] == image_files[i]].EncodedPixels
        mask = np.zeros((768, 768))
        for m in all_masks:
            mask += rle_decode(m)
        
        axes[0, i].imshow(img)
        axes[0, i].axis('off')

        axes[1, i].imshow(mask)
        axes[1, i].axis('off')

    plt.show()

display_masks(train_v2_path, csv_file=train_segmentations, num_images=5)

In [ ]:
images_with_ships = train_segmentations[train_segmentations.EncodedPixels.notna()].ImageId.nunique()
images_without_ships = train_segmentations[train_segmentations.EncodedPixels.isna()].ImageId.nunique()

print(f'Number of images with ships    - {images_with_ships}  | {round(images_with_ships / train_segmentations.ImageId.nunique() * 100)}%')
print(f'Number of images without ships - {images_without_ships} | {round(images_without_ships / train_segmentations.ImageId.nunique() * 100)}%')

In [ ]:
# Create subplots
fig, axes = plt.subplots(1, 2, figsize=(15, 4))

# Plot bar chart
axes[0].bar(['No ships', 'Ships'], [images_without_ships, images_with_ships], color=plt.cm.viridis([0.2, 0.8]))
axes[0].set_title('Distribution of Ships')
axes[0].set_ylabel('Count')
axes[0].grid(axis='y', linestyle='--', alpha=0.7)

# Plot pie chart
axes[1].pie([images_without_ships, images_with_ships], labels=['No ships', 'Ships'], autopct='%1.1f%%',
            startangle=90, colors=plt.cm.viridis([0.2, 0.8]), wedgeprops={"linewidth": 1, "edgecolor": "white"})
axes[1].set_title('Percentage of Ships')
axes[1].set_aspect('equal')  # Equal aspect ratio ensures that pie is drawn as a circle

# Display the plot
plt.show()

In [ ]:
n_ships_df = train_segmentations.dropna().groupby('ImageId').count()
n_ships_df.rename({'EncodedPixels': 'n_ships'}, axis='columns', inplace=True)


In [ ]:
plt.figure(figsize=(10, 4))
plt.title('Number of Ships Distribution')
plt.xlabel('Number of Ships')
plt.ylabel('Count')
plt.bar(range(1, 16), n_ships_df.n_ships.value_counts(), color=plt.cm.viridis([0.2, 0.8]));

In [ ]:
n_ships_df.n_ships.value_counts()

# Train Test - Split Data 

In [ ]:
def split_data(data, empty_masks=2000, test_size=0.3, random_state=42):
    """
    Parameters:
    - data (DataFrame): The input DataFrame containing the dataset.
    - empty_masks (int, optional): The number of images with empty masks. Defaults to 2000.
    - test_size (float, optional): The proportion of the dataset to include in the test split. Defaults to 0.3.
    - random_state (int, optional): Seed for random number generation to ensure reproducibility. Defaults to 42.

    Returns: The training and testing sets.
    """

    masks_df = data.copy()

    # Create binary labels for the presence of ships in each image. Count the number of ships in each image.
    masks_df['ship'] = masks_df['EncodedPixels'].map(lambda c_row: 1 if isinstance(c_row, str) else 0)
    masks_df['n_ships'] = masks_df.groupby('ImageId')['ship'].transform('sum')
    masks_df.drop_duplicates(subset='ImageId', keep='first', inplace=True)

    # Keep only n empty masks
    empty_masks_df = masks_df[masks_df.ship == 0]
    masks_df = masks_df[masks_df.ship == 1]
    masks_df = pd.concat([masks_df, empty_masks_df.sample(n=empty_masks, random_state=random_state)], axis=0)

    # Stratified split based on the number of ships in each image
    train_ids, test_ids = train_test_split(masks_df, test_size=test_size, stratify=masks_df['n_ships'].values,
                                           random_state=random_state)

    train_data = data[data['ImageId'].isin(train_ids.ImageId)]
    test_data = data[data['ImageId'].isin(test_ids.ImageId)]

    return train_data, test_data


train_data, val_data = split_data(train_segmentations, empty_masks=2000, test_size=0.2)

print(f'Number of masks in train data - {train_data.shape[0]}')
print(f'Number of masks in test data - {val_data.shape[0]}')

In [ ]:
class CustomDataGenerator(Sequence):
    def __init__(self, image_folder, csv_file, batch_size=32, image_size=(768, 768)):
        self.image_folder = image_folder
        self.batch_size = batch_size
        self.image_size = image_size
        self.data = csv_file

    def __len__(self):
        return int(len(self.data) / self.batch_size)

    def __getitem__(self, index):
        batch_data = self.data[index * self.batch_size:(index + 1) * self.batch_size]

        X = []
        y = []

        for _, row in batch_data.iterrows():
            image_path = os.path.join(self.image_folder, row['ImageId'])

            # Load image
            img = cv2.imread(image_path)
            img = cv2.resize(img, self.image_size)
            img = img / 255.0  # Normalize

            # Decode RLE to mask
            all_masks = self.data[self.data['ImageId'] == row['ImageId']].EncodedPixels
            mask = np.zeros(self.image_size)
            for m in all_masks:
                decoded_mask = rle_decode(m)
                mask += cv2.resize(decoded_mask, self.image_size)

            mask = mask.astype(float)

            X.append(img)
            y.append(mask)

        return np.array(X), np.array(y)

In [ ]:
batch_size = 16
image_size = (256, 256)

train_generator = CustomDataGenerator(image_folder=train_v2_path,
                                      csv_file=train_data,
                                      batch_size=batch_size,
                                      image_size=image_size)

val_generator = CustomDataGenerator(image_folder=train_v2_path,
                                     csv_file=val_data,
                                     batch_size=batch_size,
                                     image_size=image_size)

# U-Net Model Definition

In [ ]:
from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, MaxPool2D, Conv2DTranspose, Concatenate, Input
from tensorflow.keras.models import Model


def conv_block(input, num_filters):
    x = Conv2D(num_filters, 3, padding="same")(input)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    x = Conv2D(num_filters, 3, padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    return x


def encoder_block(input, num_filters):
    x = conv_block(input, num_filters)
    p = MaxPool2D((2, 2))(x)
    return x, p


def decoder_block(input, skip_features, num_filters):
    x = Conv2DTranspose(num_filters, (2, 2), strides=2, padding="same")(input)
    x = Concatenate()([x, skip_features])
    x = conv_block(x, num_filters)
    return x


def build_unet(input_shape=(256, 256, 3)):
    inputs = Input(input_shape)

    s1, p1 = encoder_block(inputs, 64)
    s2, p2 = encoder_block(p1, 128)
    s3, p3 = encoder_block(p2, 256)
    s4, p4 = encoder_block(p3, 512)

    b1 = conv_block(p4, 1024)

    d1 = decoder_block(b1, s4, 512)
    d2 = decoder_block(d1, s3, 256)
    d3 = decoder_block(d2, s2, 128)
    d4 = decoder_block(d3, s1, 64)

    outputs = Conv2D(1, 1, padding="same", activation="sigmoid")(d4)

    model = Model(inputs, outputs, name="U-Net")
    return model


# Create the UNet model
model = build_unet()

# Display the model summary
model.summary()

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

reduceLROnPlate = ReduceLROnPlateau(monitor='val_loss', factor=0.33, patience=1, verbose=1, mode='min', min_delta=0.0001, cooldown=0, min_lr=1e-8)
early_stopping = EarlyStopping(monitor="val_loss", mode="min", verbose=0, patience=10)

callbacks_list = [reduceLROnPlate, early_stopping]

In [ ]:
def dice_coefficient(y_true, y_pred, smooth=1e-5):
    intersection = K.sum(K.abs(y_true * y_pred), axis=-1)
    return (2. * intersection + smooth) / (K.sum(K.square(y_true), -1) + K.sum(K.square(y_pred), -1) + smooth)

In [ ]:
optimizer=tf.optimizers.Adam(learning_rate=0.001)
loss=tf.losses.binary_crossentropy

model.compile(optimizer=optimizer, loss=loss, metrics=[dice_coefficient])

In [ ]:
epochs = 20
steps_per_epoch = len(train_generator)

history = model.fit(train_generator, validation_data=val_generator, epochs=epochs, steps_per_epoch=steps_per_epoch, callbacks=callbacks_list, shuffle=True, verbose=1)

In [ ]:
plt.figure(figsize=(16, 5))

# Plot training and validation loss
plt.subplot(1, 2, 1)
plt.plot(range(epochs), history.history['loss'], 'bo-', label='Training loss')
plt.plot(range(epochs), history.history['val_loss'], 'ro-', label='Validation loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

# Plot training and validation dice coefficient
plt.subplot(1, 2, 2)
plt.plot(range(epochs), history.history['dice_coefficient'], 'bo-', label='Training Dice Coefficient')
plt.plot(range(epochs), history.history['val_dice_coefficient'], 'ro-', label='Validation Dice Coefficient')
plt.title('Training and Validation Dice Coefficient')
plt.xlabel('Epochs')
plt.ylabel('Dice Coefficient')
plt.legend();

# Submission

In [ ]:
def make_submission(folder_path, model):
    list_of_images = os.listdir(folder_path)
    image_id = []
    encoded_pixels = []
    
    for img_name in list_of_images:
        # Obtaining the model prediction.
        img = preprocess_input(os.path.join(folder_path, img_name))
        mask = model.predict(img, verbose=0)
        mask = np.squeeze(mask, axis=(0, 3))
        mask = cv2.resize(mask, (768, 768))
        mask = (mask > 0.3).astype(int)
        
        if np.all(mask == 0):
            image_id.append(img_name)
            encoded_pixels.append('')
        else:
            # Apply morphological operation to distinguish individual objects
            labeled_mask = label(mask)
            for region in regionprops(labeled_mask):
                # Create a mask for the current object
                single_ship_mask = (labeled_mask == region.label).astype(np.uint8)

                # Obtain RLE for the mask
                rle = rle_encode(single_ship_mask)

                # Add values to the lists
                image_id.append(img_name)
                encoded_pixels.append(rle)
    
    # Create a DataFrame
    df = pd.DataFrame({"ImageId": image_id, "EncodedPixels": encoded_pixels})
    return df

In [ ]:
folder_path = '/kaggle/input/airbus-ship-detection/test_v2'
submission = make_submission(folder_path, model)
submission.to_csv('submission.csv', index=False)
submission.head()

In [ ]:
img_path = os.path.join(folder_path, img_name)
img = np.squeeze(preprocess_input(img_path), axis=0)
mask = model.predict(preprocess_input(img_path), verbose=0)
mask = (mask > 0.3).astype(int)

# Set the figure size
plt.figure(figsize=(15, 10))  # Adjust the width and height as needed

# Row 1
plt.subplot(2, 3, 1)
plt.imshow(img)
plt.title('Original Image')

plt.subplot(2, 3, 2)
plt.imshow(np.squeeze(mask, axis=0))
plt.title('Model Output')

plt.subplot(2, 3, 3)
plt.imshow(label(np.squeeze(mask, axis=0)))
plt.title('Labeled Output')

# Row 2
plt.subplot(2, 3, 4)
plt.imshow(rle_decode(one_sample_masks.EncodedPixels.iloc[0]))
plt.title('Decoded Mask 1')

plt.subplot(2, 3, 5)
plt.imshow(rle_decode(one_sample_masks.EncodedPixels.iloc[1]))
plt.title('Decoded Mask 2')

plt.subplot(2, 3, 6)
plt.imshow(rle_decode(one_sample_masks.EncodedPixels.iloc[2]))
plt.title('Decoded Mask 3')

# Show the plot
plt.show()